In [ ]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv("us.env"))  # read local .env file

In [ ]:
from pathlib import Path
from tqdm import tqdm
import sys

sys.path.append(Path("..").resolve().as_posix())

## Prepare LLMs

In [ ]:
from src.generator_builder import LLMConfig

llms = LLMConfig.from_azure_deployments(
    light="gpt-35-turbo-1106",
    heavy="gpt-4-1106",
    embedding="text-embedding-ada-002",
    api_version="2023-07-01-preview",
)
from llama_index.core import Settings

Settings.embed_model = llms.embed_model
Settings.llm = llms.llamaindex_light

## Assemble the RAG

In [ ]:
from src.index_builder import build_index

index, full_nodes_dict = build_index(
    persist_path = Path("index_storage").resolve(),
    collection_name = "dev1",
    lib_path = Path("../../docs_md").resolve()
)

In [ ]:
from src.generator_builder import build_generator

generator = build_generator(
    index=index, full_nodes_dict=full_nodes_dict, llm_config=llms
)

In [ ]:
response = generator.chat(message="Tell me about a basic select", chat_history=[])

print(response.response)

In [ ]:
from llama_index.core.base.llms.types import ChatMessage, MessageRole

response = generator.chat(
    chat_history=[
        ChatMessage(
            role=MessageRole.USER,
            content="Tell me about a basic select",
        ),
        ChatMessage(
            role=MessageRole.ASSISTANT,
            content="A basic `select` in EdgeDB is a command used to retrieve or compute a set of values from the database. \
            It can be used to select primitive values, objects, or computed results.",
        ),
    ],
    message="What about insert?",
)

print(response.response)

## Run benchmarks

In [ ]:
easy_query_path = Path("benchmarks/tutorial_benchmark.txt").resolve()
# hard_query_path = Path("benchmarks/user_benchmark.txt").resolve()
hard_query_path = Path("benchmarks/new_hard.txt").resolve()

In [ ]:
with easy_query_path.open("r") as f:
    easy_queries = [query.strip() for query in f.readlines()]

with hard_query_path.open("r") as f:
    hard_queries = f.read().split(">>>SEPARATOR<<<")

In [ ]:
len(hard_queries)

In [ ]:
from src.eval import PydanticResponse, run_queries

In [ ]:
from datetime import datetime

timestamp = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
responses = run_queries(query_engine, hard_queries[147:], Path(f"eval_new_hard_{timestamp}.jsonl").resolve())